In [ ]:
def hrv_per_epoch(ecg_signal, rem_epochs, sf):
    rows = []
    for i, (t0, t1) in enumerate(rem_epochs):
        a, b = int(t0*sf), int(t1*sf)
        x = ecg_signal[a:b]

        try:
            signals, info = nk.ecg_process(x, sampling_rate=sf)
            # mean HR from ECG_Rate (bpm)
            hr_mean = float(np.nanmean(signals["ECG_Rate"]))

            # HRV metrics (time domain) from detected R-peaks
            hrv = nk.hrv_time(info, sampling_rate=sf).iloc[0].to_dict()

            rows.append({
                "epoch": i,
                "start_s": float(t0),
                "hr_mean_bpm": hr_mean,
                "rmssd_ms": float(hrv.get("HRV_RMSSD", np.nan)),
                "sdnn_ms": float(hrv.get("HRV_SDNN", np.nan)),
                "pnn50_pct": float(hrv.get("HRV_pNN50", np.nan)),
                "n_beats": int(len(info["ECG_R_Peaks"])) if "ECG_R_Peaks" in info else np.nan,
                "ok": True
            })
        except Exception as e:
            rows.append({"epoch": i, "start_s": float(t0), "ok": False, "error": str(e)})

    return pd.DataFrame(rows)

ecg_signal = raw.get_data(picks=ecg_chs)[0]
sf = raw.info["sfreq"]
hrv_df = hrv_per_epoch(ecg_signal, rem_epochs, sf)
hrv_df.head()